   # End-to-End Diabetes Machine Learning                                    Pipeline I


In [1]:
# 1. Exploratory Data Analysis
# 2. Data Preprocessing & Feature Engineering
# 3. Base Models
# 4. Automated Hyperparameter Optimization
# 5. Stacking & Ensemble Learning
# 6. Prediction for a New Observation
# 7. Pipeline Main Function

resarch kısmında çeşitli geliştirmeler yapacağız, kararlar vereceğiz.
pipelinekısmında ise temel modellerin başarı skorlarını göreceğiz.

genellikle projelerde 2 dosya olur. biri çalışma dosyasıdır (research). Diğeri ise pipeline. pipelineda çalışma dosyasında olanlar scriptlere çevrildiği yerdir. son nihai hali alır. burada veri görselleştiem adımları, işte şunlara bakayım gözlemleyeyim gibi adımlar pipeline da olmaz. araştırma gözlemleme kısmı resarch dosyasında yapılır.pipeline da sadece fonksiyonalar kodlar kalır.

In [2]:
import joblib
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler


## 1. Exploratory Data Analysis


In [3]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [4]:
def cat_summary(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show(block=True)


In [5]:

def num_summary(dataframe, numerical_col, plot=False):
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[numerical_col].describe(quantiles).T)
    
    if plot:
        dataframe[numerical_col].hist(bins=20)
        plt.xlabel(numerical_col)
        plt.title(numerical_col)
        plt.show(block=True)

In [6]:
def target_summary_with_num(dataframe, target, numerical_col):
    print(dataframe.groupby(target).agg({numerical_col: "mean"}), end="\n\n\n")


In [7]:
def target_summary_with_cat(dataframe, target, categorical_col):
    print(pd.DataFrame({"TARGET_MEAN": dataframe.groupby(categorical_col)[target].mean()}), end="\n\n\n")


In [8]:
def correlation_matrix(df, cols):
    fig = plt.gcf()
    fig.set_size_inches(10, 8)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    fig = sns.heatmap(df[cols].corr(), annot=True, linewidths=0.5, annot_kws={'size': 12}, linecolor='w', cmap='RdBu')
    plt.show(block=True)


In [9]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: listMİN
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    # print(f"Observations: {dataframe.shape[0]}")
    # print(f"Variables: {dataframe.shape[1]}")
    # print(f'cat_cols: {len(cat_cols)}')
    # print(f'num_cols: {len(num_cols)}')
    # print(f'cat_but_car: {len(cat_but_car)}')
    # print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

buraya kadar yazdığımız fonksiyonları pipeline kısmına almayacağız. grab_col_names hariç.

In [10]:
df  = pd.read_csv("C:/Users/esman/OneDrive/Masaüstü/MachineLearning_datasets/diabetes.csv")
df


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/esman/OneDrive/Masaüstü/MachineLearning_datasets/diabetes.csv'

In [ ]:
#eksik değer 0 gözüküyor. bu imkansız. şuan bunu görmezden gelicez. 
check_df(df)


In [ ]:
# Değişken türlerinin ayrıştırılması
cat_cols, num_cols, cat_but_car = grab_col_names(df, cat_th=5, car_th=20)

In [ ]:
# Kategorik değişkenlerin incelenmesi
for col in cat_cols:
    cat_summary(df, col)

In [ ]:
# Sayısal değişkenlerin incelenmesi
df[num_cols].describe().T

In [ ]:
#sayısal değerleir görselleştirmek istediğimde
for col in num_cols:
    num_summary(df, col, plot=True)

In [ ]:

# Sayısal değişkenkerin birbirleri ile korelasyonu
correlation_matrix(df, num_cols)

In [ ]:
# Target ile sayısal değişkenlerin incelemesi
for col in num_cols:
    target_summary_with_num(df, "Outcome", col)

## 2. Data Preprocessing & Feature Engineering


In [ ]:
def outlier_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [ ]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [ ]:
def check_outlier(dataframe, col_name, q1=0.25, q3=0.75):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name, q1, q3)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

In [ ]:
def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

In [ ]:
df.head()


In [ ]:
# Değişken isimleri büyütmek
df.columns = [col.upper() for col in df.columns]


                          Yeni değişkenler üretme

In [ ]:
# Glucose verisini kullanarak new_glucose_cat değişkeni oluşturduk.
#yeni glucose değişkeni oluştu diye gidip glucose değerini silme.
#yeni oluşturudğun değişken anlamsız da olabilir. 
#modellleme basamağına kadar bekle. glucose silme.
df['NEW_GLUCOSE_CAT'] = pd.cut(x=df['GLUCOSE'], bins=[-1, 139, 200], labels=["normal", "prediabetes"])


In [ ]:
# Age
df.loc[(df['AGE'] < 35), "NEW_AGE_CAT"] = 'young'
df.loc[(df['AGE'] >= 35) & (df['AGE'] <= 55), "NEW_AGE_CAT"] = 'middleage'
df.loc[(df['AGE'] > 55), "NEW_AGE_CAT"] = 'old'


In [ ]:
# BMI
df['NEW_BMI_RANGE'] = pd.cut(x=df['BMI'], bins=[-1, 18.5, 24.9, 29.9, 100],
                             labels=["underweight", "healty", "overweight", "obese"])


In [ ]:
# BloodPressure
df['NEW_BLOODPRESSURE'] = pd.cut(x=df['BLOODPRESSURE'], bins=[-1, 79, 89, 123], labels=["normal", "hs1", "hs2"])


In [ ]:
check_df(df)


In [ ]:
cat_cols, num_cols, cat_but_car = grab_col_names(df, cat_th=5, car_th=20)


In [ ]:

for col in cat_cols:
    cat_summary(df, col)

In [ ]:
for col in cat_cols:
    target_summary_with_cat(df, "OUTCOME", col)

In [ ]:
#outcome bağımlı değişkenim bunu cat_cols dan çıkarıyorum.
cat_cols = [col for col in cat_cols if "OUTCOME" not in col]


Makine öğrenmesi uygulayabilmek için;
- değişkenleri ya binary encode edeceğiz.
- ya da değişkenleri sayısal formda ifade etmelisin. 

örneğin:
- oluşturduğun sayısal değişkenleri 1-2-3 gibi temsil edip olduğu gibi bırakabilirsin. bu sınıflar zaten büüyklük küçüklük kavramını içinde taşıyacak. böylece sayısal değişkenleri label encoding ile dönüştürebiliriz bu bir tercih.
- başka bir tercih olarak da bu değişkenleri one hot encoderdan geçirebiliriz. one hot encoderdan geçirilince bir değişkendeki sınıf sayısı kadar yeni kategorik değişken oluyor olacak. böylece bunlar binary encode edilmiş olacak.


drop_first=True yapılır ki 2 sınıflı olan kategorik değişkenlerin ilk sınıfını dışarıda bırakır. böylece dummy değişkenı tuzağından kurtulmuş oluruz.

In [ ]:
#one hot encoderi label encoder olarak da kullanabilmek için drop_first=true

df = one_hot_encoder(df, cat_cols, drop_first=True)

In [ ]:
check_df(df)

In [ ]:
df.columns = [col.upper() for col in df.columns]

In [ ]:
# Son güncel değişken türlerimi tutuyorum. çünkü bir sürü değişken türettim.
cat_cols, num_cols, cat_but_car = grab_col_names(df, cat_th=5, car_th=20)

In [ ]:
#cat_cols bir daha güncelle.
cat_cols = [col for col in cat_cols if "OUTCOME" not in col]

iqr hesabındaki q1=0.05 , q3=0.95 olarak giriyorum. iqr iyidir ama çok değişkenli etkiyi göz önünde bulunduramaz. bu yüzden aşırı aykırı olanlara ucundan dokunuyorum sadece.

In [ ]:
#aykırı değerleri değerlendir.
for col in num_cols:
    print(col, check_outlier(df, col, 0.05, 0.95))

In [ ]:
replace_with_thresholds(df, "INSULIN")

In [ ]:
for col in num_cols:
    print(col, check_outlier(df, col, 0.05, 0.95))

In [ ]:
# Standartlaştırma
#x_scaled numerik değişnkenlerin standartlaştırılmış halini nparray olarak tutuyor.
X_scaled = StandardScaler().fit_transform(df[num_cols])

In [ ]:
#X_scaled dataframe atıyorum.
df[num_cols] = pd.DataFrame(X_scaled, columns=df[num_cols].columns)

In [ ]:
y = df["OUTCOME"]

In [ ]:
X = df.drop(["OUTCOME"], axis=1)

In [ ]:
check_df(X)

In [ ]:
def diabetes_data_prep(dataframe):
    dataframe.columns = [col.upper() for col in dataframe.columns]

    # Glucose
    dataframe['NEW_GLUCOSE_CAT'] = pd.cut(x=dataframe['GLUCOSE'], bins=[-1, 139, 200], labels=["normal", "prediabetes"])

    # Age
    dataframe.loc[(dataframe['AGE'] < 35), "NEW_AGE_CAT"] = 'young'
    dataframe.loc[(dataframe['AGE'] >= 35) & (dataframe['AGE'] <= 55), "NEW_AGE_CAT"] = 'middleage'
    dataframe.loc[(dataframe['AGE'] > 55), "NEW_AGE_CAT"] = 'old'

    # BMI
    dataframe['NEW_BMI_RANGE'] = pd.cut(x=dataframe['BMI'], bins=[-1, 18.5, 24.9, 29.9, 100],
                                        labels=["underweight", "healty", "overweight", "obese"])

    # BloodPressure
    dataframe['NEW_BLOODPRESSURE'] = pd.cut(x=dataframe['BLOODPRESSURE'], bins=[-1, 79, 89, 123],
                                            labels=["normal", "hs1", "hs2"])

    cat_cols, num_cols, cat_but_car = grab_col_names(dataframe, cat_th=5, car_th=20)

    cat_cols = [col for col in cat_cols if "OUTCOME" not in col]

    df = one_hot_encoder(dataframe, cat_cols, drop_first=True)

    df.columns = [col.upper() for col in df.columns]

    cat_cols, num_cols, cat_but_car = grab_col_names(df, cat_th=5, car_th=20)

    cat_cols = [col for col in cat_cols if "OUTCOME" not in col]

    replace_with_thresholds(df, "INSULIN")

    X_scaled = StandardScaler().fit_transform(df[num_cols])
    df[num_cols] = pd.DataFrame(X_scaled, columns=df[num_cols].columns)

    y = df["OUTCOME"]
    X = df.drop(["OUTCOME"], axis=1)

    return X, y


In [ ]:
#veri setimin ilk halini gözlemliyorum.
df  = pd.read_csv("C:/Users/esman/OneDrive/Masaüstü/MachineLearning_datasets/diabetes.csv")
df

In [ ]:
check_df(df)


In [ ]:
#yazdığım fonksiyonu çağrıyorum.
X, y = diabetes_data_prep(df)


In [ ]:
#kontrol işlemini gerçekleştir.
check_df(X)


# 3. BASE MODELS
Bu aşama pipelineda da yer alabilir. ancak research kısmında yer alması daha doğru.
temel bazı modellere bakılır. modellerden probleme en uygun olan 1 ya da birden fazla model seçilir. bu modellerin üzerinde hiperparametre optimizasyonları gerçekleştiriliyor olunur.
genellikle LİGHTGBM en iyi model olur.


In [ ]:
#temel modellerdeki hatalara bakmka için bir fonksiyonn yazıyorum.
def base_models(X, y, scoring="roc_auc"):
    print("Base Models....")
    #sol tarafdakiler kendi isimlendirmemiz.
    classifiers = [('LR', LogisticRegression()),
                   ('KNN', KNeighborsClassifier()),
                   ("SVC", SVC()),
                   ("CART", DecisionTreeClassifier()),
                   ("RF", RandomForestClassifier()),
                   ('Adaboost', AdaBoostClassifier()),
                   ('GBM', GradientBoostingClassifier()),
                   ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
                   ('LightGBM', LGBMClassifier()),
                   #catboost okumak zor diye commentledik ama bunuda aç.
                   # ('CatBoost', CatBoostClassifier(verbose=False))
                   ]

    for name, classifier in classifiers:
        #3 katlı cross-validation geçir.
        cv_results = cross_validate(classifier, X, y, cv=3, scoring=scoring)
        print(f"{scoring}: {round(cv_results['test_score'].mean(), 4)} ({name}) ")

In [ ]:
base_models(X, y, scoring="accuracy")
#burada en başrılısı LR gözüküyor. ancak gelişmiş modellerle ilgili işlem yapmadığımız için bunun doğruluğundan çok da emin olamıyoruz.

In [ ]:
#scoring bölümünü değiştir. modellerin accuracy değerleirni getirsin.
#accuracy değerlerine göre başarıyı değerlendir.
base_models(X, y, scoring="accuracy")


In [ ]:
#f2 score göre başarıyı değerlendir.
base_models(X, y, scoring="f1")


# 4. Automated Hyperparameter Optimization
Birden fazla modelin birden fazla arama görevini fonksiyon tanımlayarak otomatik bir şekilde gerçekleştireceğiz.
arama yapmak istediğim hiperparametre setlerini giriyorum


In [ ]:
knn_params = {"n_neighbors": range(2, 50)}


In [ ]:
cart_params = {'max_depth': range(1, 20),
               "min_samples_split": range(2, 30)}

In [ ]:
rf_params = {"max_depth": [8, 15, None],
             "max_features": [5, 7, "auto"],
             "min_samples_split": [15, 20],
             "n_estimators": [200, 300]}

In [ ]:
xgboost_params = {"learning_rate": [0.1, 0.01],
                  "max_depth": [5, 8],
                  "n_estimators": [100, 200]}

In [ ]:
lightgbm_params = {"learning_rate": [0.01, 0.1],
                   "n_estimators": [300, 500]}

In [ ]:
#modeller ve aranacak olan hiperparametreler bir arada olsun istiyorum.
classifiers = [('KNN', KNeighborsClassifier(), knn_params),
               ("CART", DecisionTreeClassifier(), cart_params),
               ("RF", RandomForestClassifier(), rf_params),
               ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), xgboost_params),
               ('LightGBM', LGBMClassifier(), lightgbm_params)]


In [ ]:
def hyperparameter_optimization(X, y, cv=3, scoring="roc_auc"):
    print("Hyperparameter Optimization....")
    best_models = {}
    for name, classifier, params in classifiers:
        print(f"########## {name} ##########")
        cv_results = cross_validate(classifier, X, y, cv=cv, scoring=scoring)
        
        #bir modelin cv den önceki hatası
        print(f"{scoring} (Before): {round(cv_results['test_score'].mean(), 4)}")
       
        #hiperparametre değerlerini bul
        gs_best = GridSearchCV(classifier, params, cv=cv, n_jobs=-1, verbose=False).fit(X, y)
        final_model = classifier.set_params(**gs_best.best_params_)

        cv_results = cross_validate(final_model, X, y, cv=cv, scoring=scoring)
        
        #modelin cv den sonraki hatası
        print(f"{scoring} (After): {round(cv_results['test_score'].mean(), 4)}")
        
        #raporlama kısmı
        print(f"{name} best params: {gs_best.best_params_}", end="\n\n")
        
        best_models[name] = final_model
    return best_models

In [ ]:
best_models = hyperparameter_optimization(X, y)

# 5. Stacking & Ensemble Learning
Temeli birden fazla modeli bir arada kullanmaya dayanmaktadır.
Stacking&Ensemble Learning: topluluk öğrenme yöntemi kapsamındadır. birden fazla algortmanın bir araya gelerek bir işi yapmasına denir.
meta learning veya voiting  de denir.
birden fazla model bir iş için tahminde bulunur. ardından bunlar bir araya getirilir. ve bir gözlem birimi geldiğinde bunların hepsi birlikte o gözlme birimini tahmin etmeye çalışır.
böylece tahmin performansının artmasını bekleriz.

voting_classifier dokümantasyonuna git. 
    - Soft voiting --> sınıf gerçekleşme olasılıkları üzerinden bir oylama yapılır. en sık tekrar eden tahmin sınıfı hangisiyse onun sonucunu tahmin diye verir.
    - Hard voiting --> ön tanımlı değeri hard. en fazla oyu olan sınıfı ifade eder. ve bu sınıfı tahmin eder.

In [ ]:

def voting_classifier(best_models, X, y):
    #fonksiyonun çalıştığını görmek için:
    print("Voting Classifier...")
    #daha önce oluşturduğumuz best model içinden modelleri çağrıyorum.
    voting_clf = VotingClassifier(estimators=[('KNN', best_models["KNN"]),
                                              ('RF', best_models["RF"]),
                                              ('LightGBM', best_models["LightGBM"])],
                                  voting='soft').fit(X, y)

    cv_results = cross_validate(voting_clf, X, y, cv=3, scoring=["accuracy", "f1", "roc_auc"])
    print(f"Accuracy: {cv_results['test_accuracy'].mean()}")
    print(f"F1Score: {cv_results['test_f1'].mean()}")
    print(f"ROC_AUC: {cv_results['test_roc_auc'].mean()}")
    return voting_clf

voting_clf = voting_classifier(best_models, X, y)


# 6. Prediction for a New Observation
voiting classifier kullanarak tahmin işlemimizi yapalım.
veri seti içinden rastgele bir gözlem birimi seçip bunu tahmin edeceğiz.

In [ ]:
#değişkenlerimiz. veri ön işleme işlemlerinden sonraki son halimiz X DİR. Bu nednele x.columns.
X.columns


In [ ]:
#rastgele kullanıcı seç
random_user = X.sample(1, random_state=45)
##

In [ ]:
#bu kullanıcı için diyabet tahminleri
voting_clf.predict(random_user)

In [ ]:
#kullandığın modeli kaydet
joblib.dump(voting_clf, "voting_clf2.pkl")

In [ ]:
#kaydettiğin model çalışıyor mu bunu test etmek için modeli yükle:
new_model = joblib.load("voting_clf2.pkl")


In [ ]:
#yüklediğin modelle tahmin işlemlerini gerçekleştir.
new_model.predict(random_user)